In [28]:
import nltk
from nltk.corpus import brown
from nltk.corpus import stopwords
from collections import Counter
import string

In [29]:
nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     /Users/williamearley/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [30]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/williamearley/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
brown_corpus = brown.words()

In [32]:
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

In [33]:
filtered_corpus = [word.lower() for word in brown_corpus if word.lower() not in stop_words and word.lower() not in punctuation]

In [35]:
word_counts = Counter(filtered_corpus)

In [43]:
word_counts = dict(sorted(word_counts.items(), key=lambda item: item[1], reverse=True))

In [49]:
bad = list(word_counts.keys())[:3]
for key in bad:
    del word_counts[key]

In [51]:
list(word_counts.keys())[:3]

['one', 'would', 'said']

In [ ]:
# Building lists of 5000 and 1000 most popular words.

In [52]:
V = list(word_counts.keys())[:5000]

In [53]:
C = list(word_counts.keys())[:1000]

In [56]:
# Building n(w,c) = # of times c occurs in a window around w.

In [71]:
spot = V[5]

In [72]:
spot_index = filtered_corpus.index(spot)

In [73]:
spot_index

708

In [69]:
V[:5]

['one', 'would', 'said', 'new', 'could']

In [66]:
filtered_corpus[:15]

['fulton',
 'county',
 'grand',
 'jury',
 'said',
 'friday',
 'investigation',
 "atlanta's",
 'recent',
 'primary',
 'election',
 'produced',
 '``',
 'evidence',
 "''"]

In [67]:
filtered_corpus[spot-2:spot+3]

['recent', 'primary', 'election', 'produced', '``']

In [57]:
context_window_counts = {key: 0 for key in C}

In [74]:
for w in V:
    w_index = filtered_corpus.index(w)
    if w in filtered_corpus[(w_index-2):(w_index+3)]:
        context_window_counts[w] += 1
        

KeyError: 'proper'

In [75]:
context_window_size = 2

In [76]:
n_wc_counts = Counter()
n_w_counts = Counter()
n_c_counts = Counter()

In [78]:
for i, w in enumerate(filtered_corpus):
    w_index = i
    context_window = filtered_corpus[max(0, w_index - context_window_size):w_index] + \
                     filtered_corpus[w_index + 1:w_index + context_window_size + 1]
    
    n_w_counts[w] += 1
    
    for context_word in context_window:
        if context_word in C:
            n_wc_counts[(w, context_word)] += 1
            n_c_counts[context_word] += 1

In [79]:
n_wc_counts

Counter({('fulton', 'county'): 6,
         ('grand', 'county'): 1,
         ('grand', 'said'): 1,
         ('jury', 'county'): 1,
         ('jury', 'said'): 11,
         ('friday', 'said'): 6,
         ('investigation', 'said'): 1,
         ('investigation', 'recent'): 1,
         ("atlanta's", 'recent'): 1,
         ("atlanta's", 'primary'): 1,
         ('recent', 'primary'): 1,
         ('primary', 'recent'): 1,
         ('election', 'recent'): 2,
         ('election', 'primary'): 2,
         ('produced', 'primary'): 1,
         ('produced', 'evidence'): 1,
         ('``', 'evidence'): 10,
         ("''", 'evidence'): 7,
         ("''", 'took'): 33,
         ('irregularities', 'evidence'): 1,
         ('irregularities', 'took'): 1,
         ('irregularities', 'place'): 1,
         ('took', 'place'): 18,
         ('place', 'took'): 18,
         ('place', 'said'): 5,
         ('jury', 'took'): 2,
         ('jury', 'place'): 1,
         ('said', 'place'): 5,
         ('term-end', 'said'

In [80]:
pr_wc = {(w, c): count / n_w_counts[w] for (w, c), count in n_wc_counts.items()}
pr_c = {c: count / sum(n_c_counts.values()) for c, count in n_c_counts.items()}

In [81]:
pr_wc

{('fulton', 'county'): 0.35294117647058826,
 ('grand', 'county'): 0.020833333333333332,
 ('grand', 'said'): 0.020833333333333332,
 ('jury', 'county'): 0.014925373134328358,
 ('jury', 'said'): 0.16417910447761194,
 ('friday', 'said'): 0.1,
 ('investigation', 'said'): 0.0196078431372549,
 ('investigation', 'recent'): 0.0196078431372549,
 ("atlanta's", 'recent'): 0.25,
 ("atlanta's", 'primary'): 0.25,
 ('recent', 'primary'): 0.00558659217877095,
 ('primary', 'recent'): 0.010416666666666666,
 ('election', 'recent'): 0.025974025974025976,
 ('election', 'primary'): 0.025974025974025976,
 ('produced', 'primary'): 0.011111111111111112,
 ('produced', 'evidence'): 0.011111111111111112,
 ('``', 'evidence'): 0.0011316057485572027,
 ("''", 'evidence'): 0.0007964501080896575,
 ("''", 'took'): 0.0037546933667083854,
 ('irregularities', 'evidence'): 0.125,
 ('irregularities', 'took'): 0.125,
 ('irregularities', 'place'): 0.125,
 ('took', 'place'): 0.04225352112676056,
 ('place', 'took'): 0.03157894736

In [83]:
V[:5]

['one', 'would', 'said', 'new', 'could']

In [84]:
C[:5]

['one', 'would', 'said', 'new', 'could']

In [87]:
import numpy as np
vec = np.zeros(5)

In [88]:
vec

array([0., 0., 0., 0., 0.])

In [ ]:
for word in V:
    c_dim_vec = np.zeros(len(C))
    key = word

In [93]:
ppmi_values = {}
for (w, c), count_wc in n_wc_counts.items():
    ppmi_values[(w, c)] = max(0, np.log2(count_wc / (n_w_counts[w] * pr_c[c])))

In [94]:
ppmi_values

{('fulton', 'county'): 9.072967120592226,
 ('grand', 'county'): 4.990504960400253,
 ('grand', 'said'): 1.3269267394223687,
 ('jury', 'county'): 4.509378270663637,
 ('jury', 'said'): 4.305231668323049,
 ('friday', 'said'): 3.589961145256163,
 ('investigation', 'said'): 1.2394638981720294,
 ('investigation', 'recent'): 4.693021941099922,
 ("atlanta's", 'recent'): 8.365447283071418,
 ("atlanta's", 'primary'): 9.264300559614517,
 ('recent', 'primary'): 3.7804847823502614,
 ('primary', 'recent'): 3.7804847823502614,
 ('election', 'recent'): 5.0986607423765165,
 ('election', 'primary'): 5.997514018919616,
 ('produced', 'primary'): 4.772447463284843,
 ('produced', 'evidence'): 3.6849846220345035,
 ('``', 'evidence'): 0.3894248444583177,
 ("''", 'evidence'): 0,
 ("''", 'took'): 1.0574642479030152,
 ('irregularities', 'evidence'): 7.176837718364179,
 ('irregularities', 'took'): 6.114553440109836,
 ('irregularities', 'place'): 5.69444495128357,
 ('took', 'place'): 4.129660332500044,
 ('place', '

In [95]:
phi_vectors = {}
for w in V:
    phi_vector = np.zeros(len(C))
    for i, c in enumerate(C):
        ppmi_value = ppmi_values.get((w,c), 0)
        phi_vector[i] = max(0, ppmi_value)
    phi_vectors[w] = phi_vector

In [97]:
from sklearn.decomposition import TruncatedSVD

ppmi_matrix = np.zeros((len(V), len(C)))

for i, w in enumerate(V):
    for j, c in enumerate(C):
        ppmi_matrix[i, j] = ppmi_values.get((w, c), 0)
        
svd = TruncatedSVD(n_components=100)
word_embeddings = svd.fit_transform(ppmi_matrix)

In [98]:
for i, w in enumerate(V):
    psi_w = word_embeddings[i]

In [101]:
from sklearn.metrics.pairwise import cosine_similarity

selected_words = V[50:75]

In [102]:
len(selected_words)

25

In [103]:
nearest_neighbors = {}

for word in selected_words:
    
    similarities = cosine_similarity([word_embeddings[V.index(word)]], word_embeddings)[0]
    
    neighbors = [(V[i], similarities[i]) for i in np.argsort(similarities)[::-1] if V[i] != word]
    
    nearest_neighbors[word] = neighbors[0]

In [104]:
for word, neighbor in nearest_neighbors.items():
    print(f"Word: {word}, Nearest Neighbor: {neighbor[0]}, Cosine Similarity: {neighbor[1]:.4f}")

Word: right, Nearest Neighbor: way, Cosine Similarity: 0.8710
Word: used, Nearest Neighbor: use, Cosine Similarity: 0.8583
Word: take, Nearest Neighbor: get, Cosine Similarity: 0.8654
Word: three, Nearest Neighbor: two, Cosine Similarity: 0.8927
Word: states, Nearest Neighbor: united, Cosine Similarity: 0.9169
Word: house, Nearest Neighbor: went, Cosine Similarity: 0.8452
Word: use, Nearest Neighbor: used, Cosine Similarity: 0.8583
Word: without, Nearest Neighbor: even, Cosine Similarity: 0.8288
Word: place, Nearest Neighbor: one, Cosine Similarity: 0.8504
Word: american, Nearest Neighbor: national, Cosine Similarity: 0.7687
Word: around, Nearest Neighbor: back, Cosine Similarity: 0.8997
Word: however, Nearest Neighbor: also, Cosine Similarity: 0.8598
Word: home, Nearest Neighbor: going, Cosine Similarity: 0.8073
Word: small, Nearest Neighbor: large, Cosine Similarity: 0.7860
Word: found, Nearest Neighbor: since, Cosine Similarity: 0.8066
Word: mrs., Nearest Neighbor: mr., Cosine Simil

In [105]:
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

num_clusters = 100

In [106]:
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_assignments = kmeans.fit_predict(word_embeddings)

/Users/williamearley/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [107]:
distances = euclidean_distances(word_embeddings, kmeans.cluster_centers_)
coherence_scores = 1 / (1 + distances)

In [109]:
best_clusters_indices = coherence_scores.argmax(axis=1)
best_clusters_words = {i: [V[j] for j in np.argsort(coherence_scores[i])[::-1][:5]] for i in best_clusters_indices}


In [111]:
for cluster_index, words in best_clusters_words.items():
    print(f"Cluster {cluster_index + 1} - Top words: {', '.join(words)}")

Cluster 11 - Top words: part, first, man, less, home
Cluster 43 - Top words: man, might, part, found, water
Cluster 63 - Top words: water, first, head, less, man
Cluster 65 - Top words: found, man, could, part, less
Cluster 99 - Top words: less, make, long, states, use
Cluster 71 - Top words: found, man, something, new, people
Cluster 87 - Top words: world, something, left, us, place
Cluster 28 - Top words: might, man, water, part, found
Cluster 48 - Top words: found, man, something, head, could
Cluster 14 - Top words: found, something, man, new, course
Cluster 38 - Top words: head, since, enough, less, people
Cluster 91 - Top words: found, place, man, something, thought
Cluster 88 - Top words: less, made, never, hand, make
Cluster 9 - Top words: head, found, man, could, people
Cluster 85 - Top words: home, part, water, man, might
Cluster 80 - Top words: found, place, something, used, af
Cluster 29 - Top words: man, new, found, course, number
Cluster 59 - Top words: man, less, part, co